In [9]:
import cv2
import numpy as np 
import imutils

# Advanced Contour Properties
Contour properties can take you a long, long way when building computer vision applications, especially when you are first getting started. It takes a bit of creative thinking and a lot of discipline not to jump to more advanced techniques such as machine learning and training your own object classifier — but by paying attention to contour properties we can actually perform object identification for simple objects quite easily.

Let’s go ahead and get started reviewing some of the more advanced contour properties, beginning with aspect ratio.

# Aspect Ratio
The first “advanced” contour property we’ll discuss is the aspect ratio. The aspect ratio is actually not that complicated at all, hence why I’m putting the term “advanced” in quotations. But despite its simplicity, it can be very powerful.

For example, I’ve personally used aspect ratio to distinguish between squares and rectangles and detect handwritten digits in images and prune them from the rest of the contours.

The actual definition of the contour’s aspect ratio is as follows:

aspect ratio = image width / image height

Yep. It’s really that simple. The aspect ratio is simply the ratio of the image width to the image height.

Shapes with an aspect ratio < 1 have a height that is greater than the width — these shapes will appear to be more “tall” and elongated. For example, most digits and characters on a license plate have an aspect ratio that is less than 1 (since most characters on a license plate are taller than they are wide).

And shapes with an aspect ratio > 1 have a width that is greater than the height. The license plate itself is an example of a object that will have an aspect ratio greater than 1 since the width of a physical license plate is always greater than the height.

Finally, shapes with an aspect ratio = 1 (plus or minus some \epsilon of course), have approximately the same width and height. Squares and circles are examples of shapes that will have an aspect ratio of approximately 1.

# Extent
The extent of a shape or contour is the ratio of the contour area to the bounding box area:

extent = shape area / bounding box area

Recall that the area of an actual shape is simply the number of pixels inside the contoured region. On the other hand, the rectangular area of the contour is determined by its bounding box, therefore:

bounding box area = bounding box width x bounding box height

In all cases the extent will be less than 1 — this is because the number of pixels inside the contour cannot possibly be larger the number of pixels in the bounding box of the shape.

Whether or not you use the extent when trying to distinguish between various shapes in images is entirely dependent on your application. And furthermore, you’ll have to manually inspect the values of the extent to determine which ranges are good for distinguishing between shapes — and later in this section, I’ll show you exactly how to perform this inspection.

# Convex Hull
I like to think of a convex hull as a super elastic rubber band to bundle together a bunch of mail envelopes. I can place however many envelopes I want inside this rubber band, regardless of their size. And no matter what, this super elastic rubber band will surround these envelopes and hold them together. This super elastic rubber band never leaves any extra space or any extra slack — it requires the minimum amount of space to enclose all my envelopes.

A convex hull is almost like a mathematical rubber band. More formally, given a set of X points in the Euclidean space, the convex hull is the smallest possible convex set that contains these X points.

On the left we have our original shape. And in the center we have the convex hull of original shape. Notice how the rubber band has been stretched to around all extreme points of the shape, but leaving no extra space along the contour — thus the convex hull is the minimum enclosing polygon of all points of the input shape, which can be seen on the right.

Another important aspect of the convex hull that we should discuss is the convexity. Convex curves are curves that appear to “bulge out”. If a curve is not bulged out, then we call it a convexity defect.

The gray outline of the hand in the image above is our original shape. The red line is the convex hull of the hand. And the black arrows, such as in between the fingers, are where the convex hull is “bulged in” rather than “bulged out”. Whenever a region is “bulged in”, such as in the hand image above, we call them convexity defects.

Perhaps not surprisingly, the convex hull and convexity defects play a major role in hand gesture recognition, as it allows us to utilize the convexity defects of the hand to count the number of fingers. We’ll be exploring hand gesture recognition and convexity defects in Module 13 of this course.

But hand gesture recognition is not the only thing the convex hull is good for. We also use it when computing another important contour property: solidity.

# Solidity
The last advanced contour I want to discuss is the solidity of a shape. The solidity of a shape is the area of the contour area divided by the area of the convex hull:

solidity = contour area / convex hull area

Again, it’s not possible to have a solidity value greater than 1. The number of pixels inside a shape cannot possibly outnumber the number of pixels in the convex hull, because, by definition, the convex hull is the smallest possible set of pixels enclosing the shape.

Just as in the extent of a shape, when using the solidity to distinguish between various objects you’ll need to manually inspect the values of the solidity to determine the appropriate ranges. For example (and as we’ll see in the next sub-section), the solidity of a shape is actually perfect for distinguishing between the X’s and O’s on a tic-tac-toe board.

In [24]:
image=cv2.imread("railway.jpg")
gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
contours=cv2.findContours(gray.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
cnts=imutils.grab_contours(contours)

In [30]:
for c in cnts:
    area=cv2.contourArea(c)
    box=cv2.boundingRect(c)
    hull=cv2.convexHull(c)
    hull_area=cv2.contourArea(hull)
    solidity=area/float(hull_area)
    clone=image.copy()
    cv2.drawContours(clone,[hull],-1,255,2)
cv2.imshow("Clone",clone)
cv2.waitKey(0)

-1

In [26]:
clone=image.copy()
cv2.drawContours(clone,[hull],-1,255,2)
cv2.imshow("Clone",clone)
cv2.waitKey(0)

-1

# Contour Approximation
As the name suggests, contour approximation is an algorithm for reducing the number of points in a curve with a reduced set of points — thus, an approximation. This algorithm is commonly known as the Ramer-Douglas-Peucker algorithm, or simply: the split-and-merge algorithm.

The general assumption of this algorithm is that a curve can be approximated by a series of short line segments. And we can thus approximate a given number of these line segments to reduce the number of points it takes to construct a curve.

Overall, the resulting approximated curve consists of a subset of points that were defined by the original curve.

The actual algorithm itself is already implemented in OpenCV via the cv2.approxPolyDP  function, so luckily we do not have to implement the algorithm by hand. However, the recursive algorithm is fairly straightforward and I would definitely suggest giving the excellent Wikipedia article on the approximation algorithm a quick read.

Before we get too far, I want to take a look at an example image so we can understand the utility of contour approximation. Then, after this little example image and source code, I’ll return to the contour approximation algorithm and discuss the parameters that we need to pay attention to.

Take a look at the example image below — our goal here is to detect only the rectangles, while ignoring the circles/ellipses:

In [41]:
image=cv2.imread("4.png")
gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
cnts=cv2.findContours(gray.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
cnts=imutils.grab_contours(cnts)
#cv2.imshow("Shapes",image)
#cv2.waitKey(0)

In [1]:
for c in cnts:
    box=cv2.boundingRect(c)
    #approx=cv2.approxPolyDP(c,0.01*peri,True)
    peri=cv2.arcLength(c,True)
    approx = cv2.approxPolyDP(c, 0.01 * peri, True)
    if len(approx)==4:
        cv2.drawContours(image,[c],-1,255,2)
cv2.imshow("iMAGE",image)
cv2.waitKey(0)

NameError: name 'cnts' is not defined